In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/NathanUA/BASNet.git

Cloning into 'BASNet'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 255 (delta 66), reused 57 (delta 57), pack-reused 182
Receiving objects: 100% (255/255), 3.60 MiB | 4.02 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [ ]:
import os
import cv2
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim

import numpy as np
from PIL import Image
import glob

from BASNet.data_loader import RescaleT,Rescale
from BASNet.data_loader import CenterCrop
from BASNet.data_loader import ToTensor
from BASNet.data_loader import ToTensorLab
from BASNet.data_loader import SalObjDataset

from BASNet.model import BASNet

def normPRED(d):
	ma = torch.max(d)
	mi = torch.min(d)

	dn = (d-mi)/(ma-mi)

	return dn

def save_output(image_name,pred,d_dir):

	predict = pred
	predict = predict.squeeze()
	predict_np = predict.cpu().data.numpy()

	im = Image.fromarray(predict_np*255).convert('RGB')
	img_name = image_name.split("/")[-1]
	image = io.imread(image_name)
	imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

	pb_np = np.array(imo)

	aaa = img_name.split(".")
	bbb = aaa[0:-1]
	imidx = bbb[0]
	for i in range(1,len(bbb)):
		imidx = imidx + "." + bbb[i]

	imo.save(d_dir+imidx+'.png')




In [ ]:
# path to model
model_dir = '/content/drive/MyDrive/CS Work/projet CS/models/basnet.pth'

# --------- 3. model define ---------
print("...load BASNet...")
net = BASNet(3,1)
net.load_state_dict(torch.load(model_dir))
if torch.cuda.is_available():
  net.cuda()
net.eval()

#path for differents class
general_path_img_dir = '/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final/'
general_path_img_pred = '/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final_no_bg/'
classes = os.listdir(general_path_img_dir)
classes.remove('.DS_Store')

for class_ in classes:
  print(class_)

  # generate the path
  image_dir = general_path_img_dir + class_+ "/"
  prediction_dir = general_path_img_pred + class_+ "/"
  img_name_list = glob.glob(image_dir + '*.jpg')

  print(img_name_list)
	
  # --------- 2. dataloader ---------
  #1. dataload
  test_salobj_dataset = SalObjDataset(img_name_list = img_name_list, lbl_name_list = [],transform=transforms.Compose([RescaleT(256),ToTensorLab(flag=0)]))
  test_salobj_dataloader = DataLoader(test_salobj_dataset, batch_size=1,shuffle=False,num_workers=1)



  # --------- 4. inference for each image ---------
  for i_test, data_test in enumerate(test_salobj_dataloader):

    print("inferencing:",img_name_list[i_test].split("/")[-1])

    inputs_test = data_test['image']
    inputs_test = inputs_test.type(torch.FloatTensor)

    if torch.cuda.is_available():
      inputs_test = Variable(inputs_test.cuda())
    else:
      inputs_test = Variable(inputs_test)

    d1,d2,d3,d4,d5,d6,d7,d8 = net(inputs_test)

    # normalization
    pred = d1[:,0,:,:]
    pred = normPRED(pred)

    ## reconstruct the imagewithout back ground

    # get img
    img = data_test['image'][0].detach().numpy()

    # get mask from pred
    mask = pred.detach().numpy()[0]
    #save_output(img_name_list[i_test], pred, prediction_dir)

    # thr the mask
    mask[mask >= 0.8] = 1
    mask[mask <= 0.8] = 0

    
    # get the real image with original shape
    real_img = cv2.imread(img_name_list[i_test])

    # permute
    real_img = torch.tensor(real_img).permute(2,0,1).detach().numpy()

    # reshape the mask for real image
    predict = mask
    predict = predict.squeeze()
    im = Image.fromarray(predict)
    # with the shape of the real image
    imo = im.resize((real_img.shape[2],real_img.shape[1]),resample=Image.BILINEAR)
    resize_mask = np.array(imo)

    # apply the mask
    for i in range(real_img.shape[0]):
      real_img[i] = real_img[i] * resize_mask

    # reshape image masked for save opr plot
    image_masked = torch.tensor(real_img).permute(1,2,0).detach().numpy()

    # save
    cv2.imwrite(prediction_dir+img_name_list[i_test].split("/")[-1] , image_masked)


    del d1,d2,d3,d4,d5,d6,d7,d8


...load BASNet...


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

004.Groove_billed_Ani
['/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final/004.Groove_billed_Ani/Groove_Billed_Ani_0044_1731.jpg', '/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final/004.Groove_billed_Ani/Groove_Billed_Ani_0044_1731_rt1.jpg', '/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final/004.Groove_billed_Ani/Groove_Billed_Ani_0044_1731_rt2.jpg', '/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final/004.Groove_billed_Ani/Groove_Billed_Ani_0044_1731_flipr.jpg', '/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final/004.Groove_billed_Ani/Groove_Billed_Ani_0044_1731_rdnoise.jpg', '/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final/004.Groove_billed_Ani/Groove_Billed_Ani_0044_1731_blur.jpg', '/content/drive/MyDrive/CS Work/projet CS/dataset/data_augm_train_final/004.Groove_billed_Ani/Groove_Billed_Ani_0058_1751.jpg', '/content/drive/MyDrive/CS Work/projet CS/dataset/data

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


inferencing: Groove_Billed_Ani_0044_1731_rt1.jpg
inferencing: Groove_Billed_Ani_0044_1731_rt2.jpg
inferencing: Groove_Billed_Ani_0044_1731_flipr.jpg
inferencing: Groove_Billed_Ani_0044_1731_rdnoise.jpg
inferencing: Groove_Billed_Ani_0044_1731_blur.jpg
inferencing: Groove_Billed_Ani_0058_1751.jpg
inferencing: Groove_Billed_Ani_0058_1751_rt1.jpg
inferencing: Groove_Billed_Ani_0058_1751_rt2.jpg
inferencing: Groove_Billed_Ani_0058_1751_flipr.jpg
inferencing: Groove_Billed_Ani_0058_1751_rdnoise.jpg
inferencing: Groove_Billed_Ani_0058_1751_blur.jpg
inferencing: Groove_Billed_Ani_0090_1567.jpg
inferencing: Groove_Billed_Ani_0090_1567_rt1.jpg
inferencing: Groove_Billed_Ani_0090_1567_rt2.jpg
inferencing: Groove_Billed_Ani_0090_1567_flipr.jpg
inferencing: Groove_Billed_Ani_0090_1567_rdnoise.jpg
inferencing: Groove_Billed_Ani_0090_1567_blur.jpg
inferencing: Groove_Billed_Ani_0071_1559.jpg
inferencing: Groove_Billed_Ani_0071_1559_rt1.jpg
inferencing: Groove_Billed_Ani_0071_1559_rt2.jpg
inferencing